In [ ]:
import pandas as pd
import glob

# ============================================================
# STRUTTURA DATI
# ============================================================
# data/raw_2023_2024/
#   airbnb.csv              <- Italia (Milano + Roma) 2023-2024
#   NYairbnb2023.zip        <- New York 2023  [da integrare]
#   NYairbnb2024.zip        <- New York 2024  [da integrare]
#   LosAngeles 2023-2024    <- MANCANTE
#
# data/raw_2025/
#   Milano/   listings1Milano.csv.gz, listings2Milano.csv.gz
#   Roma/     listings1Roma.csv.gz, listings2Roma.csv.gz, listings3Roma.csv.gz
#   NewYork/  listings1NewYork.csv.gz ... listings11NY.csv.gz
#   LosAngeles/ listings1LA.csv.gz ... listings4LA.csv.gz
#
# geo/
#   neighbourhoodsMilano.geojson
#   nieghborhoddsRoma.geojson
#   neighbourhoodsNY.geojson
#   neighbourhoodsLA.geojson
#   airbnb_italy_city_neighbourhoods_geojson.geojson
# ============================================================

# --- FILE 2023-2024 ---
file_italy_2024 = 'data/raw_2023_2024/airbnb.csv'

# --- FILE 2025 (raccolti automaticamente dalle cartelle) ---
files_milano_2025     = sorted(glob.glob('data/raw_2025/Milano/*.csv.gz'))
files_roma_2025       = sorted(glob.glob('data/raw_2025/Roma/*.csv.gz'))
files_newyork_2025    = sorted(glob.glob('data/raw_2025/NewYork/*.csv.gz'))
files_losangeles_2025 = sorted(glob.glob('data/raw_2025/LosAngeles/*.csv.gz'))

print("File 2025 trovati:")
for label, lst in [("Milano", files_milano_2025), ("Roma", files_roma_2025),
                   ("New York", files_newyork_2025), ("Los Angeles", files_losangeles_2025)]:
    print(f"  {label}: {len(lst)} file")

# Le colonne target essenziali che vogliamo nel nostro dataset finale
essential_cols = ['Listings id', 'City', 'Neighbourhood', 'Property type', 'Price', 'Date of scraping']


# --- FASE 1: Elaborazione dati 2023-2024 Italia ---
print("\nCaricamento dati 2023-2024 Italia...")
df_2024 = pd.read_csv(file_italy_2024)
df_2024 = df_2024[df_2024['City'].isin(['Milano', 'Roma'])]
df_2024_clean = df_2024[essential_cols].copy()


# --- FASE 2: Funzione per pulire i nuovi dati di Inside Airbnb (2025) ---
def process_inside_airbnb_files(file_list, city_name):
    dfs = []
    for f in file_list:
        print(f"  Elaborazione di {f}...")
        temp_df = pd.read_csv(f, low_memory=False)

        col_mapping = {
            'id': 'Listings id',
            'neighbourhood_cleansed': 'Neighbourhood',
            'room_type': 'Property type',
            'price': 'Price',
            'last_scraped': 'Date of scraping'
        }

        temp_df = temp_df.rename(columns=col_mapping)
        temp_df['City'] = city_name
        temp_df = temp_df[[c for c in essential_cols if c in temp_df.columns]].copy()

        if temp_df['Price'].dtype == 'object':
            temp_df['Price'] = temp_df['Price'].str.replace(r'[\$,]', '', regex=True).astype(float)

        dfs.append(temp_df)

    return pd.concat(dfs, ignore_index=True)


# --- FASE 3: Elaborazione 2025 ---
print("\nCaricamento e pulizia dati 2025...")
df_milano_2025     = process_inside_airbnb_files(files_milano_2025,     'Milano')
df_roma_2025       = process_inside_airbnb_files(files_roma_2025,       'Roma')
df_newyork_2025    = process_inside_airbnb_files(files_newyork_2025,    'New York')
df_losangeles_2025 = process_inside_airbnb_files(files_losangeles_2025, 'Los Angeles')


# --- FASE 4: Merge Finale ---
print("\nUnione dei dataset...")
df_final = pd.concat(
    [df_2024_clean, df_milano_2025, df_roma_2025, df_newyork_2025, df_losangeles_2025],
    ignore_index=True
)

# Uniformiamo le date
df_final['Date of scraping'] = pd.to_datetime(df_final['Date of scraping']).dt.date

# FILTRO: Manteniamo solo le case intere
valid_types = ['Entire home', 'Entire home/apt']
df_final = df_final[df_final['Property type'].isin(valid_types)]

df_final = df_final.sort_values(by=['City', 'Date of scraping'])

# Salvataggio
output_name = 'data/master_airbnb_24_25.csv'
df_final.to_csv(output_name, index=False)

print(f"\nFatto! Il file '{output_name}' è pronto.")
print(f"Righe totali: {len(df_final)}")
print(df_final.groupby('City').size().rename('Listings').to_string())


Caricamento dati 2024...

Caricamento e pulizia dati 2025...
Elaborazione di listings1Milano.csv.gz...
Elaborazione di listings2Milano.csv.gz...
Elaborazione di listings1Roma.csv.gz...
Elaborazione di listings2Roma.csv.gz...
Elaborazione di listings3Roma.csv.gz...

Unione dei dataset...

Fatto! Il file 'master_airbnb_italia_24_25.csv' è pronto.
Righe totali: 265101
